In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import json
from pathlib import Path
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
from datetime import datetime

def add_time_if_missing(date_str,add_time="23:59:59"):
    try:
        # 尝试解析带有时间的日期格式
        date_with_time = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        return date_str  # 已经有时间数据，返回原日期字符串
    except ValueError:
        try:
            # 尝试解析不带时间的日期格式
            date_without_time = datetime.strptime(date_str, "%Y-%m-%d")
            # 如果没有时间数据，添加当天的23:59:59
            return date_without_time.strftime("%Y-%m-%d") + f" {add_time}"
        except ValueError:
            return "日期格式不正确"

In [3]:
# 使用fuzzywuzzy进行相似度匹配
def match_company(name, choices, limit=5):
    return process.extract(name, choices, scorer=fuzz.token_sort_ratio, limit=limit)

In [4]:
issr_watch_df = pd.read_stata("./data/ISSR_WATCH.dta")
issr_watch_df['watch_datetime'] = issr_watch_df['watch_datetime'].astype(str).apply(lambda x: add_time_if_missing(x,add_time="00:00:00"))
issr_watch_df['watch_end_datetime'] = issr_watch_df['watch_end_datetime'].astype(str).apply(lambda x: add_time_if_missing(x,add_time="23:59:59"))
issr_watch_df['watch_datetime'] = pd.to_datetime(issr_watch_df['watch_datetime'])
issr_watch_df['watch_end_datetime'] = pd.to_datetime(issr_watch_df['watch_end_datetime'])
debt_watch_df = pd.read_stata("./data/DEBT_WATCH.dta")
debt_watch_df['watch_datetime'] = debt_watch_df['watch_datetime'].astype(str).apply(lambda x: add_time_if_missing(x,add_time="00:00:00"))
debt_watch_df['watch_end_datetime'] = debt_watch_df['watch_end_datetime'].astype(str).apply(lambda x: add_time_if_missing(x,add_time="23:59:59"))
debt_watch_df['watch_datetime'] = pd.to_datetime(debt_watch_df['watch_datetime'])
debt_watch_df['watch_end_datetime'] = pd.to_datetime(debt_watch_df['watch_end_datetime'])
mast_issr_df = pd.read_stata("./data/MAST_ISSR.dta")
mast_issr_num_name_dict = mast_issr_df.set_index('mast_issr_num')['issuer_nam'].to_dict()

In [4]:
def map_company(file_name:Path):
    with open(file_name, 'r') as f:
        data = json.load(f)
    company_name = data.get('Company Name',None)
    publish_date = data.get('report_publish_date',None)
    data['Company Matches'] = []
    data['Is In ISSR Watch'] = False
    data['Is In Debt Watch'] = False
    data['ISSR Watch'] = []
    data['Debt Watch'] = []
    if company_name is not None and publish_date is not None:
        matches = match_company(company_name, mast_issr_num_name_dict, limit=2)
        # print(company_name,matches)
        if len(matches) > 0:
            matches_list = pd.DataFrame(matches,columns=['Company Name','Score','Issuer Number']).to_dict(orient='records')
            data['Company Matches'] = matches_list
            max_match = matches[0]
            max_match_issr_num = max_match[-1]
            data['Max Match Company'] = matches_list[0]
            publish_date = pd.to_datetime(publish_date)
            match_issr_watch_df = issr_watch_df.query(f"mast_issr_num == {max_match_issr_num} and (watch_datetime <= '{publish_date}' or watch_datetime == 'NaT') and (watch_end_datetime >= '{publish_date}' or watch_end_datetime == 'NaT')")
            match_debt_watch_df = debt_watch_df.query(f"mast_issr_num == {max_match_issr_num} and (watch_datetime <= '{publish_date}' or watch_datetime == 'NaT') and (watch_end_datetime >= '{publish_date}' or watch_end_datetime == 'NaT')")
            if match_issr_watch_df.shape[0] > 0:
                match_issr_watch_df['watch_datetime'] = match_issr_watch_df['watch_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S').fillna('')
                match_issr_watch_df['watch_end_datetime'] = match_issr_watch_df['watch_end_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S').fillna('')
                data['ISSR Watch'] = match_issr_watch_df.to_dict(orient='records')
                data['Is In ISSR Watch'] = True
            if match_debt_watch_df.shape[0] > 0:
                match_debt_watch_df['watch_datetime'] = match_debt_watch_df['watch_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S').fillna('')
                match_debt_watch_df['watch_end_datetime'] = match_debt_watch_df['watch_end_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S').fillna('')
                data['Debt Watch'] = match_debt_watch_df.to_dict(orient='records')
                data['Is In Debt Watch'] = True
    return data

In [5]:
def get_save_path(data, matched_data_path:Path):
    is_in_issr_watch = data.get('Is In ISSR Watch',False)
    is_in_debt_watch = data.get('Is In Debt Watch',False)
    if is_in_debt_watch and is_in_issr_watch:
        save_path = matched_data_path / 'both'
    elif is_in_issr_watch:
        save_path = matched_data_path / 'issr'
    elif is_in_debt_watch:
        save_path = matched_data_path  / 'debt'
    else:
        save_path = matched_data_path  / 'none'
    save_path.mkdir(parents=True,exist_ok=True)
    save_file_path = save_path / file_path.name
    return save_file_path

In [7]:
start_year = 1995
end_year = 2000
for year in range(start_year,end_year+1):
    year_path = Path(f"./data/{year}")
    valid_file_path = year_path / 'processed' / 'valid'
    match_file_path = year_path / 'matched'
    valid_processed_file_list = list(valid_file_path.glob('*.json'))
    for file_path in valid_processed_file_list:
        exit_match_file = list(match_file_path.rglob(file_path.name))
        if len(exit_match_file) > 0:
            print(f"{year} {file_path.name} has been matched, skip it.")
            continue
        data = map_company(file_path)
        save_file_path = get_save_path(data, match_file_path)
        with open(save_file_path, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"{year} {file_path.name} matched and saved to {save_file_path}")

Geon Corporation [('Eaton Corporation', 91, 16257), ('GEICO Corporation', 91, 20385)]
Young Broadcasting, Inc. [('Young Broadcasting Inc.', 100, 12732), ('USA Broadcasting, Inc.', 86, 8028)]
Mitchell Energy & Development Corp. [('Mitchell Energy & Development Corp.', 100, 19974), ('ELECTRICAL DEVELOPMENT CO.', 74, 63069)]
Crown Central Petroleum Corporation [('Crown Central Petroleum Corporation', 100, 20393), ('Dyco Petroleum Corporation', 79, 3681)]
Mid-American Waste Systems, Inc. [('Mid-American Waste Systems, Inc.', 100, 5474), ('BioMedical Waste Systems, Inc.', 79, 35364)]
Chancellor Radio Broadcasting Company [('Chancellor Broadcasting Company', 91, 11325), ('Chancellor Broadcasting Licensee Company', 86, 12267)]
Mid-American Waste Systems, Inc. [('Mid-American Waste Systems, Inc.', 100, 5474), ('BioMedical Waste Systems, Inc.', 79, 35364)]
Granite Broadcasting [('Noble Broadcasting Group', 77, 434), ('Granite Broadcasting Corporation', 77, 15681)]
General Host Corporation [('Ge

KeyboardInterrupt: 

In [9]:


# 示例使用
date_str1 = "2024-10-22"
date_str2 = "2024-10-22 10:30:00"

print(add_time_if_missing(date_str1))  # 输出: 2024-10-22 23:59:59
print(add_time_if_missing(date_str2))  # 输出: 2024-10-22 10:30:00


2024-10-22 23:59:59
2024-10-22 10:30:00
